In [90]:
#Iniciando modulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pandas as pd
import pyodbc
import time
import re


In [5]:
#Acesso ao banco de dados
def executar_conexao_bd():
    server = 'DESKTOP-0P0H9IO'
    database = 'Dados_Pavimento'
    #username = 'gleisson.santos'
    #password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
        #'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

SystemError: <class 'pyodbc.Error'> returned a result with an error set

In [6]:
#Acesso ao banco de dados
def executar_conexao_bd():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        #'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [72]:
df = pd.read_sql('SELECT * FROM [Hds700]', executar_conexao_bd())
df = pd.DataFrame(df, columns = ['SS', 'OS','Tipo', 'Serviço','Ano'])
#df = df.astype(str)

# Filtra as linhas onde "Tipo" é igual a "26 - MANUTENCAO DE HIDROMETRO" e "Serviço" é igual a "12 - SUBSTITUICAO HID." OU "766 - REV HIDROMETRO PARADO"
df = df.loc[(df['Tipo'] == '26 - MANUTENCAO DE HIDROMETRO') & ((df['Serviço'] == '12 - SUBSTITUICAO HID.') | (df['Serviço'] == '766 - REV HIDROMETRO PARADO'))]

# Seleciona apenas a coluna "Serviço"

display(df)
n_linhas = len(df)
print("Número de linhas: ", n_linhas)



Executado a conexão com o BD......


,SS,OS,Tipo,Serviço,Ano
8854,950327746,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020
8858,950331729,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020
8861,951743416,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020
8863,953725963,1,26 - MANUTENCAO DE HIDROMETRO,766 - REV HIDROMETRO PARADO,2020
8864,953725964,1,26 - MANUTENCAO DE HIDROMETRO,766 - REV HIDROMETRO PARADO,2020
...,...,...,...,...,...
16329,978970982,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2022
16330,978971018,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2022
16331,978971045,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2022
16332,978971157,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2022


Número de linhas:  6215


In [81]:
#Inicinado automação / abrindo navegador
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\gleisson.santos\.wdm\drivers\chromedriver\win32\111.0.5563.64]
C:\Users\GLEISS~1.SAN\AppData\Local\Temp/ipykernel_3244/3751352982.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [82]:
login = 't034183'
senha = 'CNB@2022'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [83]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(2)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
arvore_acesso(driver, 'insum', 'GOSI_anchor')

In [85]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').clear()
    
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-panel"]/input').click()
    
preencher_ss(driver, '978971045', '1')

In [78]:
um_meio = ['HIDROMETRO RES MONO 0,75M3/H 1/2"S/C 45 - HD 1.5',
           'CONJ CONEXOES HIDROM 1/2 PP 1,5 M3/H',
          'REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']



de_tres = ['HIDROMETRO RES MULT 1,5M3/H 1/2" S/C 45º',
           'CONJ CONEXOES HIDROM 1/2 PP 3 E 1,5 M3/H',
          'REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']


In [89]:
campo = driver.find_element(By.XPATH, '//*[@id="form-j_idt135-j_idt147-item"]')
texto = campo.text
print(texto)


EXECUTADO A INSTALACAO DO HD A22HW0076890 LT 000 LACRE FIO DE ACO 740133 APOS O FURTO DO HD ANTIGO Y22S166802, CAIXA MURETA, LIGACAO ABERTA, IMOVEL HABITADO - EQ 127-A GENILSON


In [94]:
#Acessando e inserindo material
def campo_material(driver, material):
    campo = driver.find_element(By.XPATH, '//*[@id="form-j_idt135-j_idt147-item"]')
    texto = campo.text
    padrao = r"A2[012]\w+|Y2[012]\w+"
    resultados = re.findall(padrao, texto)
    if 'A22' or 'A20' or 'A21' in resultados:
        for item in material:
            campo1 = driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-tmpCodigoAutoComp-codAutoComp_input"]')
            campo1.click()
            time.sleep(0.4)
            campo1.clear()
            time.sleep(0.5)
            campo1.send_keys(item)
            time.sleep(2)
            campo1.send_keys(Keys.RETURN)
            time.sleep(1)
    

campo_material(driver, um_meio)

In [23]:
#Acessando e inserindo quantidade e medir
def campo_quantidade(driver, qtd, medir):
    campo2 = driver.find_element(By.XPATH, '//*[@id="form-cpnlAssociarInsumo-quantidade-item"]')
    campo2.click()
    campo2.clear()
    campo2.send_keys(qtd)
    time.sleep(0.5)
    #Acessando e inserindo Disp p medir S/N
    select_element = driver.find_element(By.ID, 'form-cpnlAssociarInsumo-j_idt290-comboBooleanSN-item')
    select = Select(select_element)
    select.select_by_value(medir)
    # select.select_by_visible_text('Não')
    time.sleep(0.5)
    #Confirar insumo
    driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-confirmar"]').click()
    
#campo_quantidade(driver, '3', 'N')

In [ ]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    #comando_sql = (f"UPDATE [Janeiro26/12-25/01] SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS={SS} ")
    comando_sql = "UPDATE [Insumar] SET Material='{}', Mureta='{}', Hd='{}' WHERE SS={}".format(
    ID.replace("'", "''"), MT.replace("'", "''"), HD.replace("'", "''"), SS)  
    cursor.execute(comando_sql)
    
cursor.commit()

In [ ]:
def geral():
    #Inicinado automação / abrindo navegador
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    chrome_options = webdriver.ChromeOptions()
    time.sleep(1)
    
    login = 't034183'
    senha = 'CNB@2022'
    driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
    time.sleep(1)
      

    # Procurar a tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(1)
    
    arvore_acesso(driver, 'insum', 'GOSI_anchor')
    time.sleep(1)
    
    itens = ['INTDIG01 - LUVA PVC JS 40MM', 'INTDIG01 - CAP PVC JS 25MM']

    
    for i, row in df.iterrows():
        preencher_ss(driver, str(row['SS']), str(row['OS']))
        time.sleep(1)
        for item in itens:
            campo_material(driver, item)            
            time.sleep(1)
            campo_quantidade(driver, '3', 'N')
            time.sleep(1)
            print("Materiais insumados com sucesso! :)")
   
    

In [28]:
def esperar_sumir(driver):
    wait = WebDriverWait(driver, 30)
    element = wait.until(EC.invisibility_of_element_located((By.ID, "j_idt24_modal")))

In [ ]:
geral()

In [25]:
def login(driver, user, senha):
    
    # Esperar pela tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(user)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(0.5)

def geral2():
    #Iniciando automação / abrindo navegador
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    chrome_options = webdriver.ChromeOptions()
    time.sleep(0.5)
    
    user = 't034183'
    senha = 'CNB@2022'
    driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
    time.sleep(0.5)
    
    # Esperar pela tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(user)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(0.5)
    
    arvore_acesso(driver, 'insum', 'GOSI_anchor')
    time.sleep(1)
    
    itens = ['INTDIG01 - LUVA PVC JS 40MM', 'INTDIG01 - CAP PVC JS 25MM']
    
    contador = 0

    for i, row in df.iterrows():
        preencher_ss(driver, str(row['SS']), str(row['OS']))
        time.sleep(1)
        for item in itens:
            campo_material(driver, item)            
            time.sleep(0.5)
            esperar_sumir(driver)
            campo_quantidade(driver, 3, 'N')
            time.sleep(0.5)
            contador += 1

    print(f"{contador} Ss(s) insumada(s) com sucesso!")



In [ ]:
geral2()

In [73]:
import tkinter as tk

def interface_grafica():
    # Cria a janela da interface gráfica
    janela = tk.Tk()
    janela.geometry("300x300")
    janela.title("Insumação de SS")

    # Cria o frame onde serão adicionados os widgets
    frame = tk.Frame(janela)
    frame.pack(pady=20)

    # Adiciona o label para o campo de itens
    label_itens = tk.Label(frame, text="Itens:")
    label_itens.pack()

    # Adiciona a caixa de entrada para os itens
    entrada_itens = tk.Entry(frame)
    entrada_itens.pack()

    # Adiciona o label para o campo de quantidade
    label_quantidade = tk.Label(frame, text="Quantidade:")
    label_quantidade.pack()

    # Adiciona a caixa de entrada para a quantidade
    entrada_quantidade = tk.Entry(frame)
    entrada_quantidade.pack()

    # Adiciona o label para o campo de medir
    label_medir = tk.Label(frame, text="Medir:")
    label_medir.pack()

    # Adiciona a caixa de seleção para o medir
    medir = tk.StringVar()
    medir.set('N')
    selecao_medir = tk.OptionMenu(frame, medir, 'S', 'N')
    selecao_medir.pack()

    # Adiciona o botão de insumação
    botao_insumar = tk.Button(frame, text="Insumar", command=geral2)
    botao_insumar.pack(pady=20)

    # Inicia a janela
    janela.mainloop()


In [74]:
interface_grafica()

NameError: name 'geral2' is not defined

In [77]:
import re

texto = "EXECUTADO A SUBSITUICAO DO HD A22HW0126338 Y22HW0126338 APOS FURTO E INSTALADO O HD NOVO A22HW0078184 LT 000 LACRE FIO DE ACO 714600. CAIXA MURETA, LIGACAO ABERTA, IMOVEL HABITADO. EQ 126-A LUIZ ANTONIO."

padrao = r"A2[012]\w+|Y2[012]\w+"

resultados = re.findall(padrao, texto)

print(resultados)


['A22HW0126338', 'Y22HW0126338', 'A22HW0078184']
